## MASK DETECTION ON PARTICULAR IMAGE:

In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import imutils
import time
import cv2
import os

- First, the input image is loaded from the disk and then, face detection is done using Caffe based face detector. The Caffe model have prototxt file(s) which define the model architecture (i.e., the layers themselves).
- Caffe model file which contains the weights for the actual layers. Finally, the prediction is done on the input image as shown in the figure.

#### Image load:

In [2]:
frame = cv2.imread('young.jpg')
frame = cv2.resize(frame,(224,224))

#### Our Mask detection model:

In [3]:
maskNet = load_model("mask_detector.model")

### 1.Using Caffe Model:

#### Pre-Trained Face-Detector Model:

In [4]:
# load our serialized face detector model from disk
prototxtPath = r"face_detector\deploy.prototxt"
weightsPath = r"face_detector\res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

#### Function to detect presence of Mask: RES10_CAFFE MODEL

In [5]:
def detect_and_predict_mask(frame, faceNet, maskNet):
	# grab the dimensions of the frame and then construct a blob
	# from it
	(h, w) = frame.shape[:2]
	blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),
		(104.0, 177.0, 123.0))

	# pass the blob through the network and obtain the face detections
	faceNet.setInput(blob)
	detections = faceNet.forward()
	print(detections.shape)

	# initialize our list of faces, their corresponding locations,
	# and the list of predictions from our face mask network
	faces = []
	locs = []
	preds = []

	# loop over the detections
	for i in range(0, detections.shape[2]):
		# extract the confidence (i.e., probability) associated with
		# the detection
		confidence = detections[0, 0, i, 2]

		# filter out weak detections by ensuring the confidence is
		# greater than the minimum confidence
		if confidence > 0.5:
			# compute the (x, y)-coordinates of the bounding box for
			# the object
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")

			# ensure the bounding boxes fall within the dimensions of
			# the frame
			(startX, startY) = (max(0, startX), max(0, startY))
			(endX, endY) = (min(w - 1, endX), min(h - 1, endY))

			# extract the face ROI, convert it from BGR to RGB channel
			# ordering, resize it to 224x224, and preprocess it
			face = frame[startY:endY, startX:endX]
# 			face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
			face = cv2.resize(face, (224, 224))
			face = img_to_array(face)
			face = preprocess_input(face)

			# add the face and bounding boxes to their respective
			# lists
			faces.append(face)
			locs.append((startX, startY, endX, endY))

	# only make a predictions if at least one face was detected
	if len(faces) > 0:
		# for faster inference we'll make batch predictions on *all*
		# faces at the same time rather than one-by-one predictions
		# in the above `for` loop
		faces = np.array(faces, dtype="float32")
		preds = maskNet.predict(faces, batch_size=32)

	# return a 2-tuple of the face locations and their corresponding
	# locations
	return (locs, preds)
            

#### Function to show predictions with image:

In [6]:
def show_predicted_img(frame,faceNet,maskNet):
    frame = imutils.resize(frame, width=400)

        # detect faces in the frame and determine if they are wearing a
        # face mask or not
    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

        # loop over the detected face locations and their corresponding
        # locations
    for (box, pred) in zip(locs, preds):
            # unpack the bounding box and predictions
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred

            # determine the class label and color we'll use to draw
            # the bounding box and text
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

            # include the probability in the label
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

            # display the label and bounding box rectangle on the output
            # frame
        cv2.putText(frame, label, (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

        # show the output frame
    cv2.imshow("Frame", frame)
    cv2.waitKey(0)


In [7]:
# show_predicted_img(frame,faceNet,maskNet)

### 2.Using Dlib face detector:

In [8]:
import dlib
from PIL import Image

In [9]:
detector = dlib.get_frontal_face_detector()


In [10]:
def rec(img):
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    dets=detector(gray,1)
    if dets is not None:
        for face in dets:
            left=face.left()
            top=face.top()
            right=face.right()
            bottom=face.bottom()
            cv2.rectangle(img,(left,top),(right,bottom),(0,255,0),2)

In [11]:

def mask(img):
    faces=[]
    image = cv2.resize(img, (224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)
    image = image
    faces.append(image)
    faces = np.array(faces, dtype="float32")
    preds = maskNet.predict(faces, batch_size=32)
    if preds[0][0]>0.5:
        result='mask'
    else:
        result='no-mask'
    print(preds)
    cv2.putText(img, result, (100,200), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.imshow('Video', img) 

In [12]:

#rec(frame)
# mask(frame)
# cv2.waitKey(0)

### 3.Using Haar_cascade:

In [13]:
face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
labels_dict={0:'MASK',1:'NO MASK'}
color_dict={0:(0,255,0),1:(0,0,255)}

In [14]:
def predict_with_haar(frame):
    faces_list =[]
    img = frame
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.1,1)
    print("hello")
    print(faces)
    
    for (x,y,w,h) in faces:
        face_img=img[y:y+w,x:x+w]
        image = cv2.resize(face_img, (224, 224))
        image = img_to_array(image)
        image = preprocess_input(image)
        faces_list.append(image)
        faces_list = np.array(faces_list, dtype="float32")
        
        preds = maskNet.predict(faces_list, batch_size=32)
        print(preds)
        label=np.argmax(preds,axis=1)[0]
        
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    cv2.imshow('LIVE',img)
    
        
        
        

In [15]:
predict_with_haar(frame)
cv2.waitKey(0)

hello
[[97 35 80 80]]
[[0.9983259  0.00167413]]


-1